In [25]:
import requests
from bs4 import BeautifulSoup

API_KEY = ""
SEARCH_ENGINE_ID = "96b297446bad847ad"
BASE_URL = "https://www.googleapis.com/customsearch/v1"

# Google search
SEARCH_TOOL = {
    "type": "function",
    "function": {
        "name": "search",
        "description": "Perform a Google search given a query string.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to be used.",
                },
            },
            "required": ["query"],
        },
    },
}


SCRAPE_TOOL = {
    "type": "function",
    "function": {
        "name": "scrape",
        "description": "Extract and return the text content from the webpage at the given URL. Only the first 2000 characters of paragraph text are returned.",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "type": "string",
                    "description": "The URL of the webpage to scrape for text content.",
                },
            },
            "required": ["url"],
        },
    },
}


def scrape(url: str) -> str:
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code != 200:
            return ""

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract the text content of the webpage
        texts = "\n".join([p.get_text().strip() for p in soup.find_all("p")])[:2000]

        return texts
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""


def parse_search_item(search_item: dict[str, any]) -> dict[str, str]:
    try:
        long_description = search_item["pagemap"]["metatags"][0]["og:description"]
    except KeyError:
        long_description = "N/A"
    title = search_item.get("title")
    description = search_item.get("snippet")
    url = search_item.get("link")
    return {
        "title": title,
        "description": description,
        "long_description": long_description,
        "url": url,
    }


def search(query: str) -> list[dict[str, str]]:
    try:
        results = []
        params = {
            "key": API_KEY,
            "cx": SEARCH_ENGINE_ID,
            "q": query,
            "start": 1,
        }
        # Make the HTTP GET request to the Custom Search JSON API
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Raise an exception for bad responses
        response = response.json()

        # Extract search results
        search_results = response.get("items", [])
        results.extend(
            [parse_search_item(search_item) for search_item in search_results[:3]]
        )
        return results

    except Exception as e:
        print(f"Error making Google search: {e}")
        return []


# Example usage
if __name__ == "__main__":
    search_query = "Susanna and the Elders"
    results = search(search_query)

    if results:
        for idx, result in enumerate(results, start=1):
            print("=" * 10, f"Result #{idx}", "=" * 10)
            print("Title:", result["title"])
            print("Description:", result["description"])
            print("Long description:", result["long_description"])
            print("URL:", result["url"], "\n")
    else:
        print("No search results found.")

    print(scrape(results[0]["url"]))

========== Result #1 ==========
Title: Susanna and the Elders (Artemisia Gentileschi, Pommersfelden ...
Description: Susanna and the Elders is a 1610 painting by the Italian Baroque artist Artemisia Gentileschi and is her earliest-known signed and dated work. ... It was one of ...
Long description: N/A
URL: https://en.wikipedia.org/wiki/Susanna_and_the_Elders_(Artemisia_Gentileschi,_Pommersfelden) 

========== Result #2 ==========
Title: Susanna and the Elders – Renaissance Reframed
Description: Oct 8, 2021 ... This discussion is going to center on the above painting: Susanna and the Elders by Artemisia Gentileschi from c. 1610.
Long description: This discussion is going to center on Susanna and the Elders by Artemisia Gentileschi from c. 1610. But for us to fully understand the painting, there are some important aspects we have to discuss …
URL: https://renaissancereframed.com/2021/10/08/susanna-and-the-elders/ 

========== Result #3 ==========
Title: Susanna (Book of Daniel) - Wikipe

In [37]:
from openai import OpenAI

client = OpenAI(api_key="key", base_url="http://localhost:11434/v1")

results = [
    "Analyzing Kasper's vision - Inside The Vatican",
    "Raffaello Sanzio's (Raphael) - Marriage of the Virgin. | Art day ...",
    "Pinacoteca di Brera (Mailand) - Aktuelle 2019 - Lohnt es sich? (Mit fotos)",
    "Moja Top-lista, czyli drugi spacer po Pinacoteca di Brera (post_71 ...",
    "「ブレラ絵画館(ブレラ美術館)」完全ガイド – チケット予約・見どころ・行き方・混みぐあい",
    "Pinacoteca di Brera (Milan, Italy): Top Tips Before You Go (with Photos ...",
    "IMG_0228 The Marriage of the Virgin (1504), painting by Ra… | Flickr",
    "A master visual catechist: Raphael and the Italian High Renaissance ...",
    "Marriage of the Virgin after Raphael 'Raffaello Sanzio da Urbino', 1483 ...",
    "The Circumcision | Marco Marzile, 1500, oil on canvas. | Brule Laker ...",
]

results = "\n".join([f"{i + 1}. {r}" for i, r in enumerate(results)])


completion = client.chat.completions.create(
    model="llama3.1",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an esteemed art history professor with expertise in "
                "identifying artworks and interpreting their content. When "
                "presented with a list of visual search results, identify the "
                "actual artwork, including its title and artist. Then, describe "
                "the specific event or moment depicted in the piece. Use the "
                "tools if necessary to gather additional information to ensure "
                "your response is accurate and complete."
            ),
        },
        {
            "role": "user",
            "content": (
                f"Here is a list of visual search results of an artwork:\n\n{results}\n\n"
                f"Identify the true artwork, then describe the specific event "
                f"or moment depicted in that artwork."
            ),
        },
    ],
    tools=[SEARCH_TOOL, SCRAPE_TOOL],
    stream=True,
)

for chunk in completion:
    print(chunk.choices[0])
    # print(chunk.choices[0].delta.content, end="")

Choice(delta=ChoiceDelta(content='{"', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)
Choice(delta=ChoiceDelta(content='name', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)
Choice(delta=ChoiceDelta(content='":', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)
Choice(delta=ChoiceDelta(content=' "', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)
Choice(delta=ChoiceDelta(content='search', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)
Choice(delta=ChoiceDelta(content='",', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)
Choice(delta=ChoiceDelta(content=' "', function_call=None, refusal=None, rol

In [18]:
results = [
    "Analyzing Kasper's vision - Inside The Vatican",
    "Raffaello Sanzio's (Raphael) - Marriage of the Virgin. | Art day ...",
    "Pinacoteca di Brera (Mailand) - Aktuelle 2019 - Lohnt es sich? (Mit fotos)",
    "Moja Top-lista, czyli drugi spacer po Pinacoteca di Brera (post_71 ...",
    "「ブレラ絵画館(ブレラ美術館)」完全ガイド – チケット予約・見どころ・行き方・混みぐあい",
    "Pinacoteca di Brera (Milan, Italy): Top Tips Before You Go (with Photos ...",
    "IMG_0228 The Marriage of the Virgin (1504), painting by Ra… | Flickr",
    "A master visual catechist: Raphael and the Italian High Renaissance ...",
    "Marriage of the Virgin after Raphael 'Raffaello Sanzio da Urbino', 1483 ...",
    "The Circumcision | Marco Marzile, 1500, oil on canvas. | Brule Laker ...",
]

results = "\n".join([f"{i + 1}. {r}" for i, r in enumerate(results)])
print(
    f"Here is a list of visual search results of an image:\n\n{results}\n\n"
    f"First determine the actual subject in the image. Once identified, "
    f"craft a succinct and compelling story about it, explaining its "
    f"historical and artistic significance to engage and educate the user."
)

Here is a list of visual search results of an image:

1. Analyzing Kasper's vision - Inside The Vatican
2. Raffaello Sanzio's (Raphael) - Marriage of the Virgin. | Art day ...
3. Pinacoteca di Brera (Mailand) - Aktuelle 2019 - Lohnt es sich? (Mit fotos)
4. Moja Top-lista, czyli drugi spacer po Pinacoteca di Brera (post_71 ...
5. 「ブレラ絵画館(ブレラ美術館)」完全ガイド – チケット予約・見どころ・行き方・混みぐあい
6. Pinacoteca di Brera (Milan, Italy): Top Tips Before You Go (with Photos ...
7. IMG_0228 The Marriage of the Virgin (1504), painting by Ra… | Flickr
8. A master visual catechist: Raphael and the Italian High Renaissance ...
9. Marriage of the Virgin after Raphael 'Raffaello Sanzio da Urbino', 1483 ...
10. The Circumcision | Marco Marzile, 1500, oil on canvas. | Brule Laker ...

First determine the actual subject in the image. Once identified, craft a succinct and compelling story about it, explaining its historical and artistic significance to engage and educate the user.
